In [18]:
import sys
from pathlib import Path
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ROOT_PATH = Path().absolute().parent.parent
if str(ROOT_PATH) not in sys.path:
    sys.path.insert(1, str(ROOT_PATH))
from frequency_response import FrequencyResponse
from legacy_frequency_response import LegacyFrequencyResponse
from constants import DEFAULT_FS, PEQ_CONFIGS

In [2]:
def rmse(fr):
    ix10k = np.argmin(np.abs(10000 - fr.frequency))
    return np.sqrt(np.mean(np.square(fr.equalization[:ix10k] - fr.parametric_eq[:ix10k])))

In [3]:
def generate_stats(opt_fn, max_n=None):
    i = 0
    df_data = []
    for fp in ROOT_PATH.joinpath('results', 'oratory1990').glob('*/*/*.csv'):
        df_data.append(list(opt_fn(fp)))
        i += 1
        if max_n is not None and i >= max_n:
            break
    return pd.DataFrame(df_data, columns=['time', 'rmse', 'max_q'])

In [19]:
def opt_legacy55(fp):
    fr = LegacyFrequencyResponse.read_from_csv(fp)
    start_time = time()
    filter_matrix, _, _ = fr.optimize_parametric_eq([5, 5], DEFAULT_FS)
    return time() - start_time, rmse(fr), np.max(filter_matrix[:, 1])

In [20]:
def opt_legacy10(fp):
    fr = LegacyFrequencyResponse.read_from_csv(fp)
    start_time = time()
    filter_matrix, _, _ = fr.optimize_parametric_eq([10], DEFAULT_FS)
    return time() - start_time, rmse(fr), np.max(filter_matrix[:, 1])

In [12]:
def opt_neo4ls4hs(fp):
    fr = FrequencyResponse.read_from_csv(fp)
    configs = [PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF'], PEQ_CONFIGS['4_PEAKING_WITH_HIGH_SHELF']]
    start_time = time()
    peqs = fr.optimize_parametric_eq(configs, fs=DEFAULT_FS)
    return time() - start_time, rmse(fr), np.max([np.max([filt.q for filt in peq.filters]) for peq in peqs])

In [16]:
def opt_neo8lshs(fp):
    fr = FrequencyResponse.read_from_csv(fp)
    configs = [PEQ_CONFIGS['8_PEAKING_WITH_SHELVES']]
    start_time = time()
    peqs = fr.optimize_parametric_eq(configs, fs=DEFAULT_FS)
    return time() - start_time, rmse(fr), np.max([np.max([filt.q for filt in peq.filters]) for peq in peqs])

### Old Optimizer with 5+5 Filters

In [21]:
generate_stats(opt_legacy55, max_n=10).describe()

,time,rmse,max_q
count,10.000000,10.000000,10.000000
mean,6.416749,2.805219,5.741501
std,1.309338,7.721808,2.220761
min,5.191178,0.224766,2.915876
25%,5.673487,0.342668,3.998378
50%,5.814792,0.371758,5.659287
75%,6.841272,0.425523,6.425676
max,8.993703,24.780475,9.648385


### New Optimizer with (4+1)+(4+1) Filters

In [14]:
generate_stats(opt_neo4ls4hs, max_n=10).describe()

,time,rmse,max_q
count,10.000000,10.000000,10.000000
mean,0.873296,0.273702,5.555356
std,0.319231,0.137673,0.978815
min,0.529119,0.116526,3.179125
25%,0.614888,0.157410,6.000000
50%,0.796178,0.238391,6.000000
75%,1.086243,0.357395,6.000000
max,1.524342,0.490985,6.000000


### Old Optimizer with 10 Filters

In [22]:
generate_stats(opt_legacy10, max_n=10).describe()

,time,rmse,max_q
count,10.000000,10.000000,10.000000
mean,3.095732,0.482258,2.574368
std,0.965291,0.118569,1.358899
min,2.333523,0.215271,0.949609
25%,2.527778,0.454823,1.758102
50%,2.652265,0.510441,2.158617
75%,3.081332,0.531473,3.453866
max,4.985473,0.633726,5.014539


### New Optimizer with 8+2 Filters

In [17]:
generate_stats(opt_neo8lshs, max_n=10).describe()

,time,rmse,max_q
count,10.000000,10.000000,10.000000
mean,2.557276,0.163827,4.614335
std,0.361645,0.079021,1.078390
min,1.709383,0.071387,2.342845
25%,2.456051,0.118308,4.125507
50%,2.742622,0.142256,4.662087
75%,2.753871,0.184459,5.283779
max,2.799628,0.327084,5.879053
